In [1]:
from pathlib import Path
BASE_DIR     = Path('/content/drive/MyDrive/structure')

def init_environment(forzar_drive=False):
  # 1) Montar Google Drive
  from google.colab import drive
  drive.mount('/content/drive', force_remount=forzar_drive)

  # 2) Rutas base
  MODELS_DIR   = BASE_DIR / 'models'

  # 3) Path y sys.path
  import sys
  sys.path.append(str(MODELS_DIR))
  sys.path.append(str(BASE_DIR))

In [ ]:
# =============================================================================
# 🚀 Notebook principal de experimentación con TensorFlow/Keras
# =============================================================================

# 1) Preparar el entorno
#    - Monta Google Drive
#    - Ajusta sys.path para encontrar tus módulos en models/ y utils/
init_environment(forzar_drive=True)

# 2) Definir experimento
#    El nombre debe coincidir con un archivo YAML en configs/experiments/
exp_names = ["< EXP >"]


for exp_name in exp_names:
  # 3) Cargar configuración
  from utils.experiment.functions import load_config, load_experiment
  cfg = load_config(exp_name)

  # 4) Repeticiones
  repeats = cfg["experiment"].get("repeats")

  for rep in range(repeats):

      #  4.1) Cargar experimento
      cfg, NNClass, params, dataset, train_data, val_data, test_idx = \
          load_experiment(exp_name, repeat_index=rep)

      #  4.1.1) Revisar que 'rep' actual no haya sido previamente ejecutado
      #   Si ya existe classification_report.json  → SALTAR
      rep_report = BASE_DIR / cfg['experiment']['output_root'] / cfg['experiment']['output_subdir'] / "reports" / "classification_report.json"
      if rep_report.exists() == True:
        print(f"[SKIP] Rep: {rep} (single‐split) → ya existe classification_report.json.")
        continue

      #  4.1.2) Instanciar y Entrenar
      model   = NNClass(cfg, **params)

      if rep == 0:
        #  4.1.3 ) Mostrar resumen de la configuración
        from utils.misc.functions import print_exp_configuration
        print(f"\n✔️ Experimento «{cfg['experiment']['name']}» cargado con éxito.\n")
        print_exp_configuration(cfg)

        #  4.1.4) Mostrar arquitectura del modelo
        print("\n📋 Arquitectura del modelo:")
        model.model.summary()

      print("\n"*5)
      print(f"\n🔄 Rep {rep+1}/{repeats}")

      #  4.1.5) Entrenamiento (o retomar desde último checkpoint)
      history = model.fit(train_data, val_data)

      #  4A.6) Análisis resultados individual
      from utils.analysis.analysis import ExperimentAnalyzer
      analyzer = ExperimentAnalyzer(
         model=model.model,
         history=history,
         test_data=test_idx,
         cfg=cfg,
         effects=dataset.get_effects("test"),
         repeat_index=rep,
         show_plots=False,
         )

      analyzer.classification_report()
      analyzer.effect_report()
      analyzer.confusion_matrix(normalize="true")
      model.cleanup_old_checkpoints()

  print("\n"*2)
  print("="*15, f" | EXPERIMENTO '{exp_name}' FINALIZADO CORRECTAMENTE  | ", "="*15)
  print("\n"*3)